## Demo notebook for fitting the model of _Brunton et. al. (2013)_

This is some boilerplate, for loading packages necessary for running the notebook

In [1]:
using Distributed
#addprocs(44);
addprocs(4);

In [2]:
#@everywhere using pulse_input_DDM
using pulse_input_DDM

┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1184


      From worker 2:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
      From worker 2:	│   exception = Required dependency HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f] failed to load from a cache file.
      From worker 2:	└ @ Base loading.jl:969
      From worker 4:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
      From worker 4:	│   exception = Required dependency HDF5 [f67ccb44-e63f-5c2f-98bd-6dc0ccc4ba2f] failed to load from a cache file.
      From worker 4:	└ @ Base loading.jl:969
      From worker 3:	┌ Warning: The call to compilecache failed to create a usable precompiled cache file for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
      From worker 3:	│   exception = Required dependency Tables [bd369af6-aec1-5ad0-b16a-f7cc5008161c] failed to load from a cache file.
      From worker 3:	└ @ Base loading

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
│   exception = Required dependency MAT [23992714-dd62-5051-b70f-ba57cb901cac] failed to load from a cache file.
└ @ Base loading.jl:969
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
│   exception = Required dependency MAT [23992714-dd62-5051-b70f-ba57cb901cac] failed to load from a cache file.
└ @ Base loading.jl:969
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
│   exception = Invalid input in module list: expected MAT.
└ @ Base loading.jl:969
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
│   exception = Invalid input in module list: expected MAT.
└ @ Ba

In [3]:
using PyPlot, JLD2, DataFrames
using LinearAlgebra, Pandas, GLM

┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/GLM/6OREG.ji for GLM [38e38edf-8417-5370-95a0-9cbb8c7f171a]
└ @ Base loading.jl:1184
┌ Warning: Module Distributions with build ID 222722722110496 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Recompiling stale cache file /Users/briandepasquale/.julia/compiled/v1.1/StatsModels/4MFnV.ji for StatsModels [3eaba693-59b7-5ba5-a881-562e759f1c8d]
└ @ Base loading.jl:1184
┌ Warning: Module DataFrames with build ID 222745133468364 is missing from the cache.
│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947


      From worker 5:	┌ Warning: Module DataFrames with build ID 222821805177773 is missing from the cache.
      From worker 5:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 5:	└ @ Base loading.jl:947
      From worker 3:	┌ Warning: Module DataFrames with build ID 222821805177773 is missing from the cache.
      From worker 3:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 3:	└ @ Base loading.jl:947
      From worker 2:	┌ Warning: Module DataFrames with build ID 222821805177773 is missing from the cache.
      From worker 2:	│ This may mean DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] does not support precompilation but is imported by a module that does.
      From worker 2:	└ @ Base loading.jl:947
      From worker 4:	┌ Warning: Module DataFrames with build ID 222821

Define dictionaries with some generative parameters, a Boolean vector of which ones to fit, and initial values.

In [4]:
#parameters of the latent model
pz_gen = Dict("generative" => vcat(1.,13.,-0.5,10.,1.,0.4,0.02), 
    "name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pd_gen = Dict("generative" => vcat(0.1,0.1), 
    "name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

## simulate choices from a model, to check that generative parameters can be found

In [5]:
#define how many trials to generate data from and the time binning scale (in seconds)
ntrials,dt = Int(0.1e4), 1e-2

(1000, 0.01)

In [6]:
#generate some simulated clicks times and trial durations
data = pulse_input_DDM.sample_clicks(ntrials,dt);

In [7]:
#simulate choices from the model, given the generative parameters
pulse_input_DDM.sampled_dataset!(data, pz_gen["generative"], pd_gen["generative"]; num_reps=1, rng=4);

In [8]:
#compute the likelihood of the data, given the generative parameters
compute_LL(pz_gen["generative"], pd_gen["generative"],data)

-402.0625068034104

In [9]:
#find the ML parameters with gradient descent
@time pz_gen, pd_gen, = optimize_model(pz_gen, pd_gen, data)

┌ Warning: some parameter(s) at lower bound. bumped it (them) up 1/4 from the lower bound.
└ @ pulse_input_DDM /Users/briandepasquale/Projects/inProgress/pulse_input_DDM.jl/src/wrapper_functions.jl:66


Iter     Function value   Gradient norm 
     0     4.928052e+02     1.508580e+02
     1     4.076504e+02     1.456861e+01
     2     4.045523e+02     1.454315e+01
     3     4.023401e+02     6.905086e+00
     4     3.991265e+02     4.025686e+00
     5     3.986615e+02     3.673119e+00
     6     3.984288e+02     5.499980e+00
     7     3.983185e+02     3.437827e+00
     8     3.982588e+02     1.381193e+00
     9     3.978124e+02     5.395541e+00
    10     3.976176e+02     4.885572e+00
    11     3.971240e+02     2.923723e+00
    12     3.968307e+02     1.792665e+00
    13     3.966490e+02     5.467875e+00
    14     3.966379e+02     3.837457e+00
    15     3.964539e+02     4.060813e+00
    16     3.963729e+02     4.051075e+00
    17     3.962524e+02     1.435666e+00
    18     3.961327e+02     6.472870e-01
    19     3.960757e+02     7.332553e-01
    20     3.960404e+02     8.336341e-01
    21     3.960058e+02     3.857234e-01
    22     3.959883e+02     7.705709e-01
    23     3.959

(Dict{String,AbstractArray{T,1} where T}("generative"=>[1.0, 13.0, -0.5, 10.0, 1.0, 0.4, 0.02],"name"=>["σ_i", "B", "λ", "σ_a", "σ_s", "ϕ", "τ_ϕ"],"lb"=>[2.22045e-16, 4.0, -5.0, 2.22045e-16, 2.22045e-16, 2.22045e-16, 2.22045e-16],"ub"=>[10.0, 100.0, 5.0, 800.0, 40.0, 2.0, 10.0],"fit"=>Bool[true, true, true, true, true, true, true],"initial"=>[2.0, 15.0, -5.0, 100.0, 2.0, 0.2, 0.005],"final"=>[5.99723, 4.0, -5.0, 27.2264, 0.000250107, 3.18059e-10, 0.0121596],"state"=>[5.99723, 4.0, -5.0, 27.2264, 0.000250107, 3.18059e-10, 0.0121596]), Dict{String,AbstractArray{T,1} where T}("generative"=>[0.1, 0.1],"name"=>["bias", "lapse"],"fit"=>Bool[true, true],"initial"=>[0.0, 0.5],"final"=>[0.051127, 0.00484711],"state"=>[0.051127, 0.00484711]), Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.6931471805599452,2.3978952727983707, ...]
 * Minimizer: [1.7912978603430192,-18.721744685800658, ...]
 * Minimum: 3.959152e+02
 * Iterations: 43
 * Convergence: true
   * |x - x'| ≤ 

In [ ]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
compute_H_CI(pz_gen, pd_gen, data, dt)

In [ ]:
#identify which ML parameters have generative parameters within the CI 
pz_gen[:within_bounds] = (pz_gen[:CI_minus] .< pz_gen[:generative]) .& (pz_gen[:CI_plus] .> pz_gen[:generative])
pd_gen[:within_bounds] = (pd_gen[:CI_minus] .< pd_gen[:generative]) .& (pd_gen[:CI_plus] .> pd_gen[:generative]);

In [ ]:
DataFrames.DataFrame(pz_gen)

In [ ]:
pd_gen

In [ ]:
#sample data from the model using the ML parameters
sampled_choices = pmap((T,L,R) -> sample_model(T,L,R,pz_gen[:final],pd_gen[:final]),
    data["T"],data["leftbups"],data["rightbups"])

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,dt),data["nT"],data["leftbups"],data["rightbups"])
ΔLR = map(x-> x[end], ΔLR);

In [ ]:
#bin the real choices and the sampled choices
nbins = 15;
conds,qcut_bins = qcut(ΔLR,nbins,labels=false,retbins=true);
conds = conds .+ 1;

frac_choice_sampled = fill!(Vector{Float64}(undef,nbins),NaN)
frac_choice = fill!(Vector{Float64}(undef,nbins),NaN)
for i = 1:nbins
    frac_choice_sampled[i] = mean(sampled_choices[conds .== i])
    frac_choice[i] = mean(data["pokedR"][conds .== i])
end

In [ ]:
#fit a GLM to the real choices and the simulated choices
GLM_dataframe = DataFrames.DataFrame(X= ΔLR, Y = data["pokedR"])
GLM_data = glm(@formula(Y ~ X), GLM_dataframe, Binomial(), LogitLink())
GLM_samplesframe = DataFrames.DataFrame(X= ΔLR, Y = sampled_choices)
GLM_samples = glm(@formula(Y ~ X), GLM_samplesframe, Binomial(), LogitLink())

In [ ]:
fig = figure(figsize=(6,6))
ax = subplot(111)

scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice, color="red", label="data")
scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_sampled, color="grey", label="sampled")

PyPlot.plot(sort(GLM_dataframe[:X]), sort(predict(GLM_data)), color="red")
PyPlot.plot(sort(GLM_samplesframe[:X]), sort(predict(GLM_samples)), color="grey")

ylabel("% poked R")
xlabel(L"\Delta{LR}")
ax[:spines]["top"][:set_color]("none") # Remove the top axis boundary
ax[:spines]["right"][:set_color]("none") # Remove the top axis boundary
legend()

## fit real data

In [ ]:
path = "/mnt/bucket/labs/brody/vdtang/P055_processed"
ratnames=["P055"]
sessids=[[17061800,18041700,16052600,16070500,16111700,18010300,18022600,17122200,
        17081200,17092600,17051900,18030500,17011800,18012200,17011500,17062500,
        17112700,18032700,16082200,18060800,18032500,17082900,17080500,17081000,
        17052300,17090800,17090900,18041500,18040500,17122000]];

In [ ]:
data = aggregate_choice_data(path, sessids, ratnames, dt)

## saving things

In [ ]:
JLD2.@save "/usr/people/briandd/Projects/pulse_input_DDM.jl/data/results/marino.jld" pz pd pzF pdF